In [106]:
# !pip install --trusted-host pypi.org --trusted-host pypi.pyhton.org --trusted-host files.pythonhosted.org -U --upgrade xlrd


In [107]:
import os
import sys 
import pandas as pd
import re
from selenium import webdriver 
import win32com.client
import time
from datetime import date
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
import xlsxwriter
import concurrent.futures
from datetime import datetime
import requests

import pdfplumber

import matplotlib.pyplot as plt
import plotly.express as px

import shutil

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import StaleElementReferenceException, TimeoutException

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
#from bs4 import BeautifulSoup
import PyPDF2
from PyPDF2 import PdfReader

import xlrd

In [108]:
source_directory = r"C:\Users\bd1266\Downloads\Data Download"
backup_directory = r"C:\Users\bd1266\Downloads\Data Download\Back Up"

In [109]:
today = datetime.now().strftime("%Y-%m-%d")


In [110]:
today

'2025-02-11'

In [111]:
def start_driver():
    options = webdriver.ChromeOptions()

    # Select desired download folder.
    download_directory = "C:\\Users\\bd1266\\Downloads\\Data Download"
    prefs = {
        "download.default_directory": download_directory,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
        "safebrowsing.disable_download_protection": True,
        "profile.default_content_setting_values.notifications": 2,  # Disable notifications
        "profile.default_content_setting_values.popups": 2  # Block pop-ups
    }
    options.add_experimental_option("prefs", prefs)
    options.add_argument("--disable-popup-blocking")  # Disable pop-up blocking
    options.add_argument("--disable-notifications")  # Disable notifications

    driver = webdriver.Chrome(options=options)
    return driver

In [112]:
# def start_driver():

#     options = webdriver.ChromeOptions()

#     #Select desired download folder.
#     download_directory = "C:\\Users\\bd1266\\Downloads\\Data Download"
#     prefs = {
#                 "download.default_directory": download_directory,
#                  "download.prompt_for_download": False,
#                  "download.directory_upgrade": True,
#                  "safebrowsing.enabled": True,
#                  "safebrowsing.disable_download_protection": True}
#     options.add_experimental_option("prefs", prefs)
#     driver = webdriver.Chrome(options=options)
#     return driver



#### Principales Variables

In [113]:
try:
    driver.quit()
except:
    print("no driver open")       

In [114]:
driver = start_driver()

In [115]:
time.sleep(5)
driver.get("https://www.bcra.gob.ar/PublicacionesEstadisticas/Principales_variables.asp")
time.sleep(5)

In [116]:
list_of_names = [
    'Reservas Internacionales del BCRA', 
    'Tipo de Cambio Mayorista',  
    'Tasa de Política Monetaria', 
    'Base monetaria - Total', 
    'Circulación monetaria', 
    'Billetes y monedas en poder del público', 
    'Efectivo en entidades financieras', 
    'Depósitos de los bancos en cta. cte.', 
    'Depósitos en efectivo en las entidades financieras', 
    'M2 privado', 
    'Inflación interanual', 
    'Inflación mensual', 
    'Inflación esperada',
    'BADLAR en pesos de bancos privados (en % n.a.)',
    'BADLAR en pesos de bancos privados (en % e.a.)',
    'Tasas de interés de las operaciones de pase activas para el BCRA',
    'Tasas de interés por préstamos entre entidades financiera privadas',
    'Tasas de interés por depósitos a 30',
    'Tasa de interés de préstamos por adelantos',
    'Tasa de interés de préstamos personales',
    'Tasa de interés para uso de la Justicia']

In [117]:
cell = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div/div[2]/div/div/div/div/table/tbody'))
)

# Get all child elements within the specified cell
child_elements = cell.find_elements(By.XPATH, './*')  # Get all direct children
time.sleep(2)

# Count the number of child elements
item_count = len(child_elements)

results =[]
for i in range(1,item_count):
    
    item_xpath = f"/html/body/div/div[2]/div/div/div/div/table/tbody/tr[{i}]"
    try:
        kpi_name = driver.find_elements(By.XPATH, item_xpath + "/td[1]/a")[0].text
        kpi_date = driver.find_elements(By.XPATH, item_xpath + "/td[2]")[0].text
        kpi_value = driver.find_elements(By.XPATH, item_xpath + "/td[3]")[0].text
        
        result_dict = {
            "Fecha" : today,
            "Variable" : kpi_name,
            "Fecha Variable" : kpi_date,
            "Valor" : kpi_value
        }
        results.append(result_dict)
        print(result_dict)
    except:
        print("not found")

df = pd.DataFrame(results)

{'Fecha': '2025-02-11', 'Variable': 'Reservas Internacionales del BCRA (en millones de dólares - cifras provisorias sujetas a cambio de valuación)', 'Fecha Variable': '06/02/2025', 'Valor': '28.743'}
not found
{'Fecha': '2025-02-11', 'Variable': 'Tipo de Cambio Minorista ($ por USD) Comunicación B 9791 - Promedio vendedor', 'Fecha Variable': '10/02/2025', 'Valor': '1.086,08'}
{'Fecha': '2025-02-11', 'Variable': 'Tipo de Cambio Mayorista ($ por USD) Comunicación A 3500 - Referencia', 'Fecha Variable': '10/02/2025', 'Valor': '1.056,38'}
not found
not found
{'Fecha': '2025-02-11', 'Variable': '   Tasa de Política Monetaria (en % n.a.)', 'Fecha Variable': '10/02/2025', 'Valor': '29,00'}
{'Fecha': '2025-02-11', 'Variable': '   Tasa de Política Monetaria (en % e.a.)', 'Fecha Variable': '10/02/2025', 'Valor': '33,63'}
{'Fecha': '2025-02-11', 'Variable': '   TAMAR en pesos de bancos privados (en % n.a.)', 'Fecha Variable': '07/02/2025', 'Valor': '30,5000'}
{'Fecha': '2025-02-11', 'Variable': '

In [118]:
pattern = '|'.join(list_of_names)
df_ppalesvariables = df[df['Variable'].str.contains(pattern, case=False, na=False)]
df_ppalesvariables

C:\Users\bd1266\AppData\Local\Temp\ipykernel_21296\367063942.py:2: UserWarning:

This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.



,Fecha,Variable,Fecha Variable,Valor
0,2025-02-11,Reservas Internacionales del BCRA (en millones...,06/02/2025,28.743
2,2025-02-11,Tipo de Cambio Mayorista ($ por USD) Comunicac...,10/02/2025,"1.056,38"
3,2025-02-11,Tasa de Política Monetaria (en % n.a.),10/02/2025,"29,00"
4,2025-02-11,Tasa de Política Monetaria (en % e.a.),10/02/2025,"33,63"
10,2025-02-11,Tasas de interés de las operaciones de pase...,11/02/2025,"33,00"
11,2025-02-11,Tasas de interés por préstamos entre entida...,07/02/2025,"33,69"
12,2025-02-11,Tasas de interés por depósitos a 30 días de...,07/02/2025,"27,00"
13,2025-02-11,Tasa de interés de préstamos por adelantos ...,07/02/2025,"35,98"
14,2025-02-11,Tasa de interés de préstamos personales,07/02/2025,"67,53"
15,2025-02-11,Tasa de interés para uso de la Justicia - Comu...,11/02/2025,"18.408,5370"


## 1.1 Rava

### 1.1 Dolar

In [119]:
try:
    driver.quit()
except:
    print("no driver open")       

In [120]:
driver = start_driver()

In [121]:
time.sleep(5)
driver.get("https://www.rava.com/")
time.sleep(15)

In [122]:


body_xpath= '/html/body/div[1]/main/div/div/div[2]/div[1]/div[2]/table/tbody'


list_of_names = ["MEP", "CCL", "MAYORISTA"]

In [123]:
cell = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, body_xpath))
)

# Get all child elements within the specified cell
child_elements = cell.find_elements(By.XPATH, './*')  # Get all direct children
time.sleep(2)
# Count the number of child elements
item_count = len(child_elements)
print(item_count)
results =[]

list_of_names

df_dolar = pd.DataFrame(columns=["Fecha", "Name", "Valor", "% dia"])

# Assuming child_elements is already defined and contains the relevant elements
for i in child_elements:
    split_values = i.text.split()

    # Ensure there are at least 3 parts before adding to DataFrame
    if len(split_values) >= 3:
        # Check if the first part is in the list_of_names
        if split_values[0] in list_of_names:
            # Check if the second part is not "LEDE" or "SENEBI"
            if split_values[1] not in ["LEDE", "SENEBI"]:
                # Create a new row as a DataFrame
                new_row = pd.DataFrame({
                    "Fecha" : today,
                    "Name": [split_values[0]],
                    "Valor": [split_values[1]],
                    "% dia": [split_values[2]]
                })
                # Append the new row to the existing DataFrame using concat
                df_dolar = pd.concat([df_dolar, new_row], ignore_index=True)
            else:
                print(f"Skipping entry with 'LEDE' or 'SENEBI': {split_values}")
        else:
            print(f"Value not useful: {split_values}")
    else:
        print(f"Not enough parts in split_values: {split_values}")

# Display the final DataFrame


10
Value not useful: ['OFICIAL', '1.076,00', '0,1', '0,2', '2,2']
Value not useful: ['TARJETA', '1.398,80', '0,1', '0,2', '2,2']
Skipping entry with 'LEDE' or 'SENEBI': ['MEP', 'LEDE', '1.174,37', '0,1', '0,5', '1,3']
Skipping entry with 'LEDE' or 'SENEBI': ['MEP', 'SENEBI', '1.183,71', '0,2', '1,5', '1,5']
Value not useful: ['OFI-MEP', '9,70', '-', '14,1', '-13,4']
Value not useful: ['OFI-CCL', '10,70', '-', '2,9', '-16,4']
Value not useful: ['MEP-CCL', '0,90', '-', '-50,0', '-35,7']


In [124]:
df_dolar

,Fecha,Name,Valor,% dia
0,2025-02-11,MAYORISTA,"1.055,25",-
1,2025-02-11,MEP,"1.181,07","-0,3"
2,2025-02-11,CCL,"1.190,65","0,6"


### 1.2 Bonos

In [125]:
try:
    driver.quit()
except:
    print("no driver open")       

In [126]:
driver = start_driver()

KeyboardInterrupt: 

In [ ]:
time.sleep(5)
driver.get("https://www.rava.com/cotizaciones/bonos")
time.sleep(15)

In [ ]:


body_xpath= '/html/body/div[1]/main/div/div/div[2]/table/tbody'


list_of_names = ["AL30", "AL30D", "AL30C", "GD30", "GD30D", "GD30C" , "BPY26"]

In [ ]:
cell = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, body_xpath))
)

# Get all child elements within the specified cell
child_elements = cell.find_elements(By.XPATH, './*')  # Get all direct children
time.sleep(2)
# Count the number of child elements
item_count = len(child_elements)
print(item_count)
results =[]

list_of_names

df_bonos = pd.DataFrame(columns=["Fecha", "Name", "Valor", "% dia"])

# Assuming child_elements is already defined and contains the relevant elements
for i in child_elements:
    split_values = i.text.split()

    # Ensure there are at least 3 parts before adding to DataFrame
    if len(split_values) >= 3:
        # Check if the first part is in the list_of_names
        if split_values[0] in list_of_names:
            
                new_row = pd.DataFrame({
                    "Fecha" : today,
                    "Name": [split_values[0]],
                    "Valor": [split_values[1]],
                    "% dia": [split_values[2]]
                })
                # Append the new row to the existing DataFrame using concat
                df_bonos = pd.concat([df_bonos, new_row], ignore_index=True)
        
        else:
            pass
    else:
        print(f"Not enough parts in split_values: {split_values}")

# Display the final DataFrame


In [ ]:
df_bonos

### 1.3 Lo mejor - Lo peor

In [ ]:
try:
    driver.quit()
except:
    print("no driver open")       

In [ ]:
driver = start_driver()

In [ ]:
time.sleep(5)
driver.get("https://www.rava.com")
time.sleep(15)

In [ ]:
body_xpath = '/html/body/div[1]/main/div/div/div[2]/div[8]/div[2]/table/tbody'


cell = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, body_xpath))
)

# Get all child elements within the specified cell
child_elements = cell.find_elements(By.XPATH, './*')  # Get all direct children
time.sleep(2)

# Count the number of child elements
item_count = len(child_elements)
print(item_count)

# Initialize an empty DataFrame with specified column names
df_lomejor = pd.DataFrame(columns=["Fecha", "Name", "Valor", "% dia"])

# Loop through each child element and extract data
for i in child_elements:
    split_values = i.text.split()

    new_row = pd.DataFrame({
        "Fecha": [today],
        "Name": [split_values[0]],
        "Valor": [split_values[1]],
        "% dia": [split_values[2]]
    })
    
    # Append the new row to the existing DataFrame using pd.concat
    df_lomejor = pd.concat([df_lomejor, new_row], ignore_index=True)

print("Lo mejor")
df_lomejor

In [ ]:
body_xpath = '/html/body/div[1]/main/div/div/div[2]/div[9]/div[2]/table/tbody'


cell = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, body_xpath))
)

# Get all child elements within the specified cell
child_elements = cell.find_elements(By.XPATH, './*')  # Get all direct children
time.sleep(2)

# Count the number of child elements
item_count = len(child_elements)
print(item_count)

# Initialize an empty DataFrame with specified column names
df_lopeor = pd.DataFrame(columns=["Fecha", "Name", "Valor", "% dia"])

# Loop through each child element and extract data
for i in child_elements:
    split_values = i.text.split()

    new_row = pd.DataFrame({
        "Fecha": [today],
        "Name": [split_values[0]],
        "Valor": [split_values[1]],
        "% dia": [split_values[2]]
    })
    
    # Append the new row to the existing DataFrame using pd.concat
    df_lopeor = pd.concat([df_lopeor, new_row], ignore_index=True)

print("Lo peor")
df_lopeor

### 1.4 Acciones Merval

In [ ]:
try:
    driver.quit()
except:
    print("no driver open")       

In [ ]:
driver = start_driver()

In [ ]:
time.sleep(5)
driver.get("https://www.rava.com/cotizaciones/acciones-argentinas")
time.sleep(15)

In [ ]:
body_xpath = '/html/body/div[1]/main/div/div/div[2]/table/tbody'


cell = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, body_xpath))
)

# Get all child elements within the specified cell
child_elements = cell.find_elements(By.XPATH, './*')  # Get all direct children
time.sleep(2)

# Count the number of child elements
item_count = len(child_elements)
print(item_count)

# Initialize an empty DataFrame with specified column names
df_merval_acciones = pd.DataFrame(columns=["Especie", "Lider", "Valor", "% dia"])

# Loop through each child element and extract data
for i in child_elements:
    split_values = i.text.split()

    new_row = pd.DataFrame({
        "Fecha": [today],
        "Especie": [split_values[0]],
        "Valor": [split_values[1]],
        "% dia": [split_values[2]]
    })
    
    # Append the new row to the existing DataFrame using pd.concat
    df_merval_acciones = pd.concat([df_merval_acciones, new_row], ignore_index=True)

#print("Lo mejor")
df_merval_acciones

In [ ]:
body_xpath = '/html/body/div[1]/main/div/div/div[4]/div[2]/div[1]/table/tbody'


cell = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, body_xpath))
)

# Get all child elements within the specified cell
child_elements = cell.find_elements(By.XPATH, './*')  # Get all direct children
time.sleep(2)

# Count the number of child elements
item_count = len(child_elements)
print(item_count)

# Initialize an empty DataFrame with specified column names
df_merval = pd.DataFrame(columns=["Especie", "Ultimo Valor", "% dia", "% mes", "% año"])

# Loop through each child element and extract data
for i in child_elements:
    split_values = i.text.split()
    if "USD" in i.text:
        new_row = pd.DataFrame({
            "Fecha": [today],
            "Especie": [split_values[0] + " " + split_values[1]],
            "Ultimo Valor": [split_values[2]],
            "% dia": [split_values[3]],
            "% mes": [split_values[4]],
            "% año": [split_values[5]],
        })
    else: 

        new_row = pd.DataFrame({
            "Fecha": [today],
            "Especie": [split_values[0]],
            "Ultimo Valor": [split_values[1]],
            "% dia": [split_values[2]],
            "% mes": [split_values[3]],
            "% año": [split_values[4]],
        })
    
    # Append the new row to the existing DataFrame using pd.concat
    df_merval = pd.concat([df_merval, new_row], ignore_index=True)

#print("Lo mejor")
df_merval

### MATBA FOREX

In [ ]:
try:
    driver.quit()
except:
    print("no driver open")       

In [ ]:
driver = start_driver()

In [ ]:
time.sleep(5)
driver.get("https://matbarofex.primary.ventures/")
time.sleep(15)

In [ ]:
body_xpath = '/html/body/div[1]/div/div[2]/div/div/div/div/div/div/div[2]/div/div[2]/div[2]'


cell = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, body_xpath))
)

# Get all child elements within the specified cell
child_elements = cell.find_elements(By.XPATH, './*')  # Get all direct children
time.sleep(2)

# Count the number of child elements
item_count = len(child_elements)
print(item_count)

In [ ]:
df_forex = pd.DataFrame(columns=[ "Instrumento", "Vol C", "Compra", "Venta", "Vol V","Fecha"])
for i in range(4):
    
    split_values = child_elements[i].text.split()

    new_row = pd.DataFrame([{
        "Instrumento": split_values[0],
        "Vol C": split_values[1],
        "Compra": split_values[2],
        "Venta": split_values[3],
        "Vol V": split_values[4],
        "Fecha": today
    }])
    #print(new_row)
    df_forex = pd.concat([df_forex, new_row], ignore_index=True)
df_forex

### Trading Economics

#### Crypto

In [ ]:
try:
    driver.quit()
except:
    print("no driver open")       

In [ ]:
driver = start_driver()

In [ ]:
time.sleep(5)
driver.get("https://tradingeconomics.com/crypto")
time.sleep(15)

In [ ]:
table = driver.find_elements(By.XPATH, '/html/body/form/div[4]/div/div/div[1]/div/table/tbody')[0]
rows = table.find_elements(By.TAG_NAME, 'tr')
data = []

# Iterate through each row and extract data
for row in rows:
    columns = row.find_elements(By.TAG_NAME, 'td')
    
    # Extract data from each column based on the specified headers
    crypto = columns[0].text.strip()
    price = columns[1].text.strip()
    day = columns[2].text.strip()
    day_percent = columns[3].text.strip()
    weekly = columns[4].text.strip()
    monthly = columns[5].text.strip()
    ytd = columns[6].text.strip()
    yoy = columns[7].text.strip()
    marketCap = columns[8].text.strip()
    date = today
    data.append([crypto, price, day, day_percent, weekly, monthly, ytd, yoy, marketCap, date])
    # Print the extracted data
    print(f"crypto: {crypto}, price: {price}, Day: {day}, day_percent: {day_percent}, Weekly: {weekly}, Monthly: {monthly}, YTD: {ytd}, YoY: {yoy}, marketCap: {marketCap},  Date: {date}")

df_trading_economics_crypto = pd.DataFrame(data, columns=['crypto', 'price', 'Day', 'day_percent', 'Weekly', 'Monthly', 'YTD', 'YoY', 'marketCap', 'Date'])

In [ ]:
df_trading_economics_crypto

In [ ]:
df_trading_economics_crypto

In [ ]:
df_dolar

In [ ]:
df_bonos

In [ ]:
df_merval

In [ ]:
df_merval_acciones

In [ ]:
df_lopeor

In [ ]:
df_lomejor

In [ ]:
df_forex

#### Bonds

In [ ]:
try:
    driver.quit()
except:
    print("no driver open")       

In [ ]:
driver = start_driver()

In [ ]:
time.sleep(5)
driver.get("https://tradingeconomics.com/bonds")
time.sleep(15)

In [ ]:
table = driver.find_elements(By.XPATH, '/html/body/form/div[4]/div/div/div[1]/div/table/tbody')[0]
rows = table.find_elements(By.TAG_NAME, 'tr')
data = []

# Iterate through each row and extract data
for row in rows:
    columns = row.find_elements(By.TAG_NAME, 'td')
    
    # Extract data from each column based on the specified headers
    major = columns[1].text.strip()
    yield_value = columns[2].text.strip()
    day = columns[3].text.strip()
    weekly = columns[4].text.strip()
    monthly = columns[5].text.strip()
    ytd = columns[6].text.strip()
    yoy = columns[7].text.strip()
    date = today
    data.append([major, yield_value, day, weekly, monthly, ytd, yoy, date])
    # Print the extracted data
    print(f"Major: {major}, Yield: {yield_value}, Day: {day}, Weekly: {weekly}, Monthly: {monthly}, YTD: {ytd}, YoY: {yoy}, Date: {date}")

df_trading_economics_bonds = pd.DataFrame(data, columns=['Major', 'Yield', 'Day', 'Weekly', 'Monthly', 'YTD', 'YoY', 'Date'])

In [ ]:
df_trading_economics_bonds

#### Stocks

In [ ]:
try:
    driver.quit()
except:
    print("no driver open")       

In [ ]:
driver = start_driver()

In [ ]:
time.sleep(5)
driver.get("https://tradingeconomics.com/stocks")
time.sleep(15)

In [ ]:
table = driver.find_elements(By.XPATH, '/html/body/form/div[4]/div/div/div[1]/div/table/tbody')[0]
rows = table.find_elements(By.TAG_NAME, 'tr')
data = []

# Iterate through each row and extract data
for row in rows:
    columns = row.find_elements(By.TAG_NAME, 'td')
    
    # Extract data from each column based on the specified headers
    major = columns[1].text.strip()
    yield_value = columns[2].text.strip()
    day = columns[3].text.strip()
    weekly = columns[4].text.strip()
    monthly = columns[5].text.strip()
    ytd = columns[6].text.strip()
    yoy = columns[7].text.strip()
    date = today
    data.append([major, yield_value, day, weekly, monthly, ytd, yoy, date])
    # Print the extracted data
    print(f"Major: {major}, Yield: {yield_value}, Day: {day}, Weekly: {weekly}, Monthly: {monthly}, YTD: {ytd}, YoY: {yoy}, Date: {date}")

df_trading_economics_stocks = pd.DataFrame(data, columns=['Major', 'Yield', 'Day', 'Weekly', 'Monthly', 'YTD', 'YoY', 'Date'])

In [ ]:
df_trading_economics_stocks

# Economia Local 

## 1- INDEC

In [ ]:
driver  = start_driver()
time.sleep(10)
driver.get("https://www.indec.gob.ar")
time.sleep(12)

###  1.1 “Tasa de Desocupación”, “Precios al Consumidor”, “Indice de producción industrial manufacturero” y “Estimador mensual de actividad económica
##### Accedemos al elemento padre y descargamos la información de sus elementos hijos.

In [ ]:
def download_values(kpi):
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div"))  # Adjust this to a more specific locator if needed
        )

        # Locate the child element containing the specific text
        child_xpath = f"//*[contains(text(), '{kpi}')]"
        child_element = driver.find_element(By.XPATH, child_xpath)

        # Locate the parent element of that child
        parent_element = child_element.find_element(By.XPATH, "..")  # .. to get the parent element

        # Extract values from the parent element or its children as needed
        child_elements = parent_element.find_elements(By.XPATH, './div')  

        # Store the values in a list or dictionary
        values = [element.text.strip() for element in child_elements]

        # Print the stored values
        print("Values stored:", values)
        return values
    except Exception as e:
        print(f"failed due to following error: {e}")
        return e

In [ ]:
lista_kpis = ['Tasa de desocupación','Precios al consumidor',
             'Índice de producción industrial manufacturero',
              'Estimador mensual de actividad económica']

In [ ]:
today_dict = {}
for kpi in lista_kpis:
    elements = download_values(kpi)
    result_dict = {kpi:{"Fecha" : today,
                        "Periodo": elements[2],
                        "Valor": elements[0]}}
    today_dict.update(result_dict)
    
today_dict


In [ ]:
df_1 = pd.DataFrame.from_dict(today_dict, orient='index')

# Resetting index, cleaner df
df_1.reset_index(inplace=True)

# Renaming columns for clarity
df_1.columns = ['Metric', 'Fecha', 'Periodo', 'Valor']

# Display the DataFrame
df_1

## 2 - Índice de precios al Consumidor

### IPC Nucleo Nacional y por sectores 
### Descargamos un csv de la base de datos del gobierno.

In [ ]:
try:
    driver.quit()
except:
    print("no driver open")       

In [ ]:
driver = start_driver()

In [ ]:
time.sleep(5)
driver.get("https://infra.datos.gob.ar/catalog/sspm/dataset/145/distribution/145.3/download/indice-precios-al-consumidor-nivel-general-base-diciembre-2016-mensual.csv")
time.sleep(12)

In [ ]:
source_directory = r"C:\Users\bd1266\Downloads\Data Download"
backup_directory = r"C:\Users\bd1266\Downloads\Data Download\Back Up"

#
file_to_rename = None
for filename in os.listdir(source_directory):
    if "indice-precios" in filename:
        file_to_rename = filename
        break  

if file_to_rename is None:
    print("No file containing 'indice-precios' found.")
else:
    
    new_filename = f"{today}_{file_to_rename}"
    old_file_path = os.path.join(source_directory, file_to_rename)
    new_file_path = os.path.join(source_directory, new_filename)
    os.rename(old_file_path, new_file_path)
    print(f"Renamed '{file_to_rename}' to '{new_filename}'.")

   

In [ ]:
indice_precios = pd.read_csv(new_file_path)

In [ ]:
indice_precios.columns

In [ ]:
indice_precios = indice_precios[['indice_tiempo', 'ipc_ng_nacional', 'ipc_ng_nacional_tasa_variacion_mensual', 'ipc_ng_nacional_tasa_variacion_acumulada_dic_ano_anterior']]

In [ ]:
indice_precios['indice_tiempo'] = pd.to_datetime(indice_precios['indice_tiempo'])

#Calculate the growth compared to the previous month
indice_precios['growth_ipc_ng_nacional_vs_lastmonth'] = indice_precios['ipc_ng_nacional'].diff()

In [ ]:
indice_precios

In [ ]:
indice_precios['ipc_ng_nacional_tasa_variacion_mensual'] = indice_precios['ipc_ng_nacional_tasa_variacion_mensual'].astype(float)

In [ ]:
max_date = indice_precios['indice_tiempo'].max()


variacion_mensual = indice_precios.loc[indice_precios['indice_tiempo'] == max_date, 'ipc_ng_nacional_tasa_variacion_mensual'].values[0] * 100
start_date_yoy = max_date - pd.DateOffset(months=11)
variacion_interanual = indice_precios.loc[(indice_precios['indice_tiempo'] >= start_date_yoy) & (indice_precios['indice_tiempo'] <= max_date), 
                                   'ipc_ng_nacional_tasa_variacion_mensual'].sum() * 100

# Step 4: Calculate year-to-date accumulated variation for the current year
variation_ytd = indice_precios.loc[indice_precios['indice_tiempo'] == max_date, 
                                    'ipc_ng_nacional_tasa_variacion_acumulada_dic_ano_anterior'].values[0] * 100

In [ ]:
start_date_yoy

#### Verificar valor de la variación interanual y su fórmula.

In [ ]:
variacion_mensual, variacion_interanual, variation_ytd

In [ ]:
data = [
    {'indicador': 'Variación Mensual', 'Valor': variacion_mensual},
    {'indicador': 'Variación Interanual', 'Valor': variacion_interanual},
    {'indicador': 'Variación YTD', 'Valor': variation_ytd}
]

# Convert the list of dictionaries to a DataFrame
df_indicadores_ipc = pd.DataFrame(data)

In [ ]:



start_date = max_date - pd.DateOffset(months=11)
filtered_df = indice_precios[(indice_precios['indice_tiempo'] >= start_date) & (indice_precios['indice_tiempo'] <= max_date)]


filtered_df['ipc_ng_nacional_tasa_variacion_mensual'] *= 100  # Convert to percentage
filtered_df['ipc_ng_nacional'] = filtered_df['ipc_ng_nacional'].pct_change() * 100  # Monthly percentage change


fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))


axes[0].bar(filtered_df['indice_tiempo'], filtered_df['ipc_ng_nacional_tasa_variacion_mensual'], color='skyblue')
axes[0].set_title('IPC Monthly Variation Rate (%)')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Percentage (%)')
axes[0].tick_params(axis='x', rotation=45)


for i in range(len(filtered_df)):
    axes[0].text(filtered_df['indice_tiempo'].iloc[i], 
                  filtered_df['ipc_ng_nacional_tasa_variacion_mensual'].iloc[i], 
                  f"{filtered_df['ipc_ng_nacional_tasa_variacion_mensual'].iloc[i]:.2f}", 
                  ha='center', va='bottom')

    
axes[1].bar(filtered_df['indice_tiempo'], filtered_df['ipc_ng_nacional'], color='lightcoral')
axes[1].set_title('IPC National Index Monthly Change (%)')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Percentage Change (%)')
axes[1].tick_params(axis='x', rotation=45)


for i in range(len(filtered_df)):
    axes[1].text(filtered_df['indice_tiempo'].iloc[i], 
                  filtered_df['ipc_ng_nacional'].iloc[i], 
                  f"{filtered_df['ipc_ng_nacional'].iloc[i]:.2f}", 
                  ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
try:
    shutil.move(new_file_path, backup_directory)
    print(f"Moved '{new_filename}' to '{backup_directory}'.")
except Exception as e:
    print(f"Error moving the file: {e}")

In [ ]:
df_indicadores_ipc

### IPC Nucleo Nacional por capítulos.
### Descargamos un csv de la base de datos del gobierno.

In [ ]:
try:
    driver.quit()
except:
    print("no driver open")       

In [ ]:
driver = start_driver()

In [ ]:
time.sleep(5)
driver.get("https://infra.datos.gob.ar/catalog/sspm/dataset/145/distribution/145.5/download/indice-precios-al-consumidor-apertura-por-capitulos-base-diciembre-2016-mensual.csv")
time.sleep(12)

In [ ]:
source_directory = r"C:\Users\bd1266\Downloads\Data Download"
backup_directory = r"C:\Users\bd1266\Downloads\Data Download\Back Up"

#
file_to_rename = None
for filename in os.listdir(source_directory):
    if "por-capitulos" in filename:
        file_to_rename = filename
        break  

if file_to_rename is None:
    print("No file containing 'indice-precios-por-capitulos' found.")
else:
    
    new_filename = f"{today}_{file_to_rename}"
    old_file_path = os.path.join(source_directory, file_to_rename)
    new_file_path = os.path.join(source_directory, new_filename)
    os.rename(old_file_path, new_file_path)
    print(f"Renamed '{file_to_rename}' to '{new_filename}'.")

   

In [ ]:
indice_precios_cap = pd.read_csv(new_file_path)

In [ ]:
indice_precios_cap

In [ ]:
indice_precios_cap['indice_tiempo'] = pd.to_datetime(indice_precios_cap['indice_tiempo'])

# Find the last date in the 'indice_tiempo' column
last_date = max(indice_precios_cap['indice_tiempo'])

# Filter for the last date and rename to national_data
national_data = indice_precios_cap[indice_precios_cap['indice_tiempo'] == last_date]


columns_to_visualize = [
    'ipc_vivienda_agua_electricidad_combustibles_nacional',
    'ipc_prendas_vestir_calzado_nacional',
    'ipc_restaurantes_hoteles_nacional',
    'ipc_salud_nacional',
    'ipc_educacion_nacional',
    'ipc_bebidas_alcoholicas_tabaco_nacional',
    'ipc_recreacion_cultura_nacional',
    'ipc_bienes_servicios_varios_nacional',
    'ipc_equipamiento_mantenimientos_hogar_nacional',
    'ipc_comunicaciones_nacional',
    'ipc_transporte_nacional', 
    'ipc_alimentos_bebidas_no_alcoholicas_nacional'
]

national_data = national_data.reset_index(drop=True)

In [ ]:
plt.figure(figsize=(12, 6))
bars = plt.bar(columns_to_visualize, national_data[columns_to_visualize].values.flatten())
plt.title(f'IPC Values on {last_date.strftime("%Y-%m-%d")}')
plt.xlabel('IPC Categories')
plt.ylabel('Values')
plt.xticks(rotation=45, ha='right')

# Adding values on top of the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, yval + 1, round(yval), ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Create a dynamic chart with Plotly
fig = px.line(indice_precios_cap,
              x='indice_tiempo',
              y=columns_to_visualize,
              title='IPC Values Over Time',
              labels={'indice_tiempo': 'Date'},
              markers=True)

# Show the figure
fig.show()

In [ ]:
national_data_full = indice_precios_cap[columns_to_visualize].copy()
variation_data = national_data_full.pct_change() * 100
national_data_variation = pd.DataFrame({
    'indice_tiempo': indice_precios_cap['indice_tiempo'][1:], # Skip first date for variation
})

# Add variation data to the new DataFrame
for col in columns_to_visualize:
    national_data_variation[col + '_variation'] = variation_data[col][1:] # Skip first row for NaN

# Reset index to clean up the DataFrame
national_data_variation.reset_index(drop=True, inplace=True)

national_data_variation

In [ ]:
max_date = national_data_variation['indice_tiempo'].max()
max_date_variation = national_data_variation[national_data_variation['indice_tiempo'] == max_date]

# Plotting for max date using Matplotlib
plt.figure(figsize=(12, 6))
bars = plt.bar(max_date_variation.columns[1:], max_date_variation.iloc[0][1:])
plt.title(f'IPC Variation (%) on {max_date.strftime("%Y-%m-%d")}')
plt.xlabel('IPC Categories')
plt.ylabel('Variation (%)')
plt.xticks(rotation=45)

# Adding values on top of the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, yval + (1 if yval >= 0 else -1), f"{yval:.2f}%", ha='center', va='bottom' if yval >= 0 else 'top')

plt.tight_layout()
plt.show()

# Create a dynamic chart with Plotly for all dates
fig = px.line(national_data_variation,
              x='indice_tiempo',
              y=[col for col in national_data_variation.columns if '_variation' in col],
              title='IPC Variation Over Time',
              labels={'indice_tiempo': 'Date'},
              markers=True)

# Show the figure
fig.show()

In [ ]:
try:
    shutil.move(new_file_path, backup_directory)
    print(f"Moved '{new_filename}' to '{backup_directory}'.")
except Exception as e:
    print(f"Error moving the file: {e}")

## 3 - Estimador Mensual de Actividad Económica

### Descargamos el csv con la información actualizada e histórica del EMAE desde la base de datos del gobierno.

In [ ]:
try:
    driver.quit()
except:
    print("no driver open")       

In [ ]:
driver = start_driver()

In [ ]:
time.sleep(5)
driver.get("https://infra.datos.gob.ar/catalog/sspm/dataset/143/distribution/143.3/download/emae-valores-anuales-indice-base-2004-mensual.csv")
time.sleep(12)

In [ ]:
source_directory = r"C:\Users\bd1266\Downloads\Data Download"
backup_directory = r"C:\Users\bd1266\Downloads\Data Download\Back Up"

#
file_to_rename = None
for filename in os.listdir(source_directory):
    if "emae" in filename:
        file_to_rename = filename
        break  

if file_to_rename is None:
    print("No file containing 'emae' found.")
else:
    
    new_filename = f"{today}_{file_to_rename}"
    old_file_path = os.path.join(source_directory, file_to_rename)
    new_file_path = os.path.join(source_directory, new_filename)
    os.rename(old_file_path, new_file_path)
    print(f"Renamed '{file_to_rename}' to '{new_filename}'.")

   

In [ ]:
emae_df = pd.read_csv(new_file_path)

In [ ]:
emae_df

In [ ]:
try:
    shutil.move(new_file_path, backup_directory)
    print(f"Moved '{new_filename}' to '{backup_directory}'.")
except Exception as e:
    print(f"Error moving the file: {e}")

## 4 - Indice de produccion industrial manufacturero

In [ ]:
try:
    driver.quit()
except:
    print("no driver open")       

In [ ]:
driver = start_driver()

In [ ]:
time.sleep(5)
driver.get("https://infra.datos.gob.ar/catalog/sspm/dataset/453/distribution/453.1/download/ipi-manufacturero.csv")
time.sleep(12)

In [ ]:
source_directory = r"C:\Users\bd1266\Downloads\Data Download"
backup_directory = r"C:\Users\bd1266\Downloads\Data Download\Back Up"

#
file_to_rename = None
for filename in os.listdir(source_directory):
    if "manufacturero" in filename:
        file_to_rename = filename
        break  

if file_to_rename is None:
    print("No file containing 'manufacturero' found.")
else:
    
    new_filename = f"{today}_{file_to_rename}"
    old_file_path = os.path.join(source_directory, file_to_rename)
    new_file_path = os.path.join(source_directory, new_filename)
    os.rename(old_file_path, new_file_path)
    print(f"Renamed '{file_to_rename}' to '{new_filename}'.")

   

In [ ]:
indice_manufacturero = pd.read_csv(new_file_path)

In [ ]:
indice_manufacturero

In [ ]:
try:
    shutil.move(new_file_path, backup_directory)
    print(f"Moved '{new_filename}' to '{backup_directory}'.")
except Exception as e:
    print(f"Error moving the file: {e}")

### 5- Relevamiento de expectativas de Mercado

In [ ]:
try:
    driver.quit()
except:
    print("no driver open")       

In [ ]:
driver = start_driver()

In [ ]:
time.sleep(5)
driver.get("https://www.bcra.gob.ar/PublicacionesEstadisticas/Relevamiento_Expectativas_de_Mercado.asp")
time.sleep(5)

In [ ]:
excel_link = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div/div[3]/div/div[1]/div[2]/p[2]/a[2]'))
)
excel_link.click()
time.sleep(10)

In [ ]:
file_to_rename = None
for filename in os.listdir(source_directory):
    if "relevamiento" in filename:
        print("its here")
        file_to_rename = filename
        break  

if file_to_rename is None:
    print("No file containing 'relevamiento' found.")
else:
    
    new_filename = f"{today}_{file_to_rename}"
    old_file_path = os.path.join(source_directory, file_to_rename)
    new_file_path = os.path.join(source_directory, new_filename)
    os.rename(old_file_path, new_file_path)
    print(f"Renamed '{file_to_rename}' to '{new_filename}'.")


In [ ]:
def extract_table2(file_path, sheet_name, start_col, end_col, start_row, end_row=None):
    # Construct the column range string
    col_range = f"{start_col}:{end_col}"
    
    # Read the specific columns and rows from the sheet
    if end_row == None:
        df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=col_range, skiprows=start_row-1)
    # Drop rows and columns that are completely empty
    else:
        nrows = end_row - start_row + 1
        df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=col_range, skiprows=start_row-1, nrows=nrows)
    df.dropna(how='all', inplace=True)
    df.dropna(axis=1, how='all', inplace=True)
    
    # Remove any leading or trailing whitespace from column names
    def is_date(string):
        try:
            pd.to_datetime(string)
            return True
        except ValueError:
            return False

# Verify if the column header is a date and format it as '2025-02-06' and convert it to text
    df.columns = [pd.to_datetime(col).strftime('%Y-%m-%d') if is_date(col) else col.strip() for col in df.columns]
    
    # Replace any remaining NaN values with an empty string
    df.fillna('', inplace=True)
    
    return df

sheet_name = 'Cuadros de resultados'
start_col = 'B'
end_col = 'M'
start_row = 4

# Extract the table
df_cuadro1 = extract_table2(new_file_path, sheet_name, start_col, end_col, start_row)
df_cuadro1.reset_index(inplace=True, drop=True)

In [ ]:
markers = [
    "Precios minoristas (IPC nivel general-Nacional; INDEC)",
    "Precios minoristas (IPC núcleo-Nacional; INDEC)",
    "Tasa de interés (TAMAR)",
    "Tipo de cambio nominal"
]

# Find the indices for the start and end markers
indices = [df_cuadro1[df_cuadro1['Unnamed: 1'] == marker].index[0] for marker in markers]

# Split the dataframe into three tables based on the indices
table1 = df_cuadro1.iloc[indices[0]:indices[1]]
table2 = df_cuadro1.iloc[indices[1]:indices[2]]
table3 = df_cuadro1.iloc[indices[2]:indices[3]]



In [ ]:
table2

In [ ]:
def corrrect_headers(table1):
    table1 = table1.reset_index(drop=True)
    header_row_index = table1[table1[table1.columns[0]] == 'Período'].index[0]
    table1.columns = table1.iloc[header_row_index]
    table1 = table1[header_row_index + 1:]
    table1 = table1.reset_index(drop=True)
    return table1

precios_minoristas_ipc_general = corrrect_headers(table1)
precios_minoristas_ipc_nucleo = corrrect_headers(table2)
tasa_interes_tamar = corrrect_headers(table3)

In [ ]:
precios_minoristas_ipc_general

In [ ]:
precios_minoristas_ipc_nucleo

In [ ]:
tasa_interes_tamar

In [ ]:
try:
    shutil.move(new_file_path, backup_directory)
    print(f"Moved '{new_filename}' to '{backup_directory}'.")
except Exception as e:
    print(f"Error moving the file: {e}")

In [ ]:
driver.quit()

# Guardamos los resultados !

In [ ]:
df_indicadores_ipc,
indice_manufacturero,
emae_df,
precios_minoristas_ipc_general,
precios_minoristas_ipc_general,
precios_minoristas_ipc_nucleo,
tasa_interes_tamar